In [76]:
import os
import psycopg2
from openai import OpenAI

# --- Set API key ---
client = OpenAI(api_key="sk-")  # Replace with your actual key

# --- Connect to Supabase Postgres ---
conn = psycopg2.connect(
    host="host", #replace with your host
    port=5432,
    dbname="postgres",
    user="user", # replace with your user
    password="password",  # Replace with your password
    sslmode="require"
)
cur = conn.cursor()
print("✅ Connected to Supabase")

# --- Function to get schema for any table ---
def get_table_schema(table_name):
    cur.execute(f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
    """, (table_name,))
    return cur.fetchall()

# --- Ask GPT to decide between SQL or Chat ---
def ask_any_question(question, table_name=None):
    table_info = ""
    table_hint = ""
    if table_name:
        schema = get_table_schema(table_name)
        table_info = "\n".join([f"- {col} ({dtype})" for col, dtype in schema])
        table_hint = f"The table name is `{table_name}`. Use it exactly in the SQL query."

    system_prompt = f"""
You are a smart assistant that can either:
1. Generate SQL if the question is about data in a table.
2. Answer generally if it’s general knowledge.

"Column names like years (e.g., 1943) must be in double quotes and included in the SELECT clause if used in ORDER BY."

If a table is provided, only use SQL if the answer depends on its content.

Table schema:
{table_info}

{table_hint}

For the user question below, reply in one of two ways:
- "SQL:" followed by a SQL query using the correct table and column names
- "ANSWER:" followed by a natural language answer
"""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    reply = response.choices[0].message.content.strip()

    # Route the response
    if reply.startswith("SQL:"):
        sql = reply[4:].strip()
        print(f"🧠 GPT SQL:\n{sql}")
        cur.execute(sql)
        return cur.fetchall()
    elif reply.startswith("ANSWER:"):
        return reply[7:].strip()
    else:
        return "🤖 Could not determine how to respond."

# --- Example 1: SQL question ---
print("📌 Q1:")
q1 = "What are the top 3 aircraft produced in 1943?"
print(ask_any_question(q1, table_name="ww2_aircraft_production"))

# --- Example 2: General question ---
print("\n📌 Q2:")
q2 = "Who manufactured the Spitfire?"
print(ask_any_question(q2))


✅ Connected to Supabase
📌 Q1:
🧠 GPT SQL:
SELECT type FROM ww2_aircraft_production ORDER BY "1943" DESC LIMIT 3;
[('Spitfire',), ('Hurricane',), ('Wellington',)]

📌 Q2:
The Spitfire was manufactured by Supermarine, a British aircraft manufacturer.
